In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy

from statsmodels.sandbox.regression.gmm import GMM
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

In [2]:
data = pd.read_stata('chile.dta')
data = data.set_index(['id','year'])

In [ ]:
ldata = data.copy()

ldata[['routput', 'totlab', 'renerg','realmats','rcapstock']] = np.log(
    ldata[['routput','totlab', 'renerg','realmats','rcapstock']])

ldata = ldata.dropna()

do a non-parametric regression with $l_{it}$, $m_{it}$, $k_{it}$ on $y_{it}$ and save residual, and fitted value

given a guess of $\beta_1$ and $\beta_2$ compute an implied omega i.e. productive shock
get wit-1 (i.e. without estimating a function)
the residuals should be exogenous
these are the moment conditions

https://www.aeaweb.org/content/file?id=3015

In [41]:
d = ldata['routput'],ldata[['totlab']],ldata[['rcapstock','realmats','renerg']]

def np_resids(y,x):
    """residuals from lasso"""
    poly = PolynomialFeatures(degree=3)
    x_poly = poly.fit_transform(x)
    clf = Ridge(alpha=0.0)
    clf.fit(x_poly, y) 
    resid = y-clf.predict(x_poly)
    return resid


stage1_resids = np_resids(ldata['routput'],ldata[['totlab','rcapstock','renerg','realmats']])

In [44]:
def stage2obj(params, stage1_resids, dta):
    y,x = dta
    rho = params[-1]
    params = params[:-1]
    
    x = sm.add_constant(x)
    xlag= x.groupby('id').shift(1)
    
    #compute the value of the control function from stage 1
    control = y - stage1_resids
    controllag = control.groupby('id').shift(1)
    
    if False:
        print np.array(y - x.mul(params,axis=1).sum(axis=1) )[1]
        print np.array(rho*(controllag- xlag.mul(params,axis=1).sum(axis=1) ))[1]
        print np.array(y - x.mul(params,axis=1).sum(axis=1) 
             - rho*(controllag- xlag.mul(params,axis=1).sum(axis=1) ) )[1]   
    
    ### compute the value of the shock###
    shock = (y - x.mul(params,axis=1).sum(axis=1) 
             - rho*(controllag- xlag.mul(params,axis=1).sum(axis=1) ) )    
    if False:
        print '-----'
        print np.array(x)[1]*np.array(y - x.mul(params,axis=1).sum(axis=1) 
             - rho*(controllag- xlag.mul(params,axis=1).sum(axis=1) ) )[1]  
    
        print np.array(xlag)[1]*np.array(y - x.mul(params,axis=1).sum(axis=1) 
             - rho*(controllag- xlag.mul(params,axis=1).sum(axis=1) ) )[1]  
    
    #####multiply to get moment condition #########
    moments = pd.concat((xlag,controllag),axis=1)
    moments = moments.mul(shock,axis=0)
    return moments.dropna()
    
d = ldata['routput'],ldata[['rcapstock','totlab']]
stage2obj(np.ones(4), stage1_resids, d)

const  rcapstock     totlab    routput
id       year                                           
18.0     1980  0.195252   2.051434   0.662472   2.081874
         1981 -0.142531  -1.483868  -0.461758  -1.533066
54.0     1980  0.248446   2.059740   0.853627   2.527602
         1981 -0.387994  -3.163360  -1.333096  -4.001083
         1982  0.265327   2.127548   0.789210   2.583670
         1983 -0.313420  -2.601060  -0.795843  -3.115470
         1984  0.095895   0.813253   0.237312   0.924306
         1985 -0.389332  -3.262285  -0.974299  -3.789595
         1986  1.228018  10.144449   3.509606  11.966046
         1987 -0.045909  -0.373876  -0.131204  -0.459534
126.0    1989 -1.269971  -8.410147  -6.206140 -14.136708
         1990 -1.126729  -9.701664  -5.108565 -12.353221
         1991  0.940070   8.564735   5.444315  11.006856
         1992  0.339324   3.093145   1.589160   3.794163
         1993  0.329543   3.001694   1.539523   3.692753
         1994  0.329246   3.003994   1.490130   3.607058
         1995 -0.533603  -4.861392  -2.447249  -5.945711
504.0    1980 -0.079236  -0.613760  -0.236208  -0.776480
         1981 -0.563023  -4.405484  -1.476387  -5.623764
         1982 -0.338754  -2.611973  -1.014419  -3.337734
         1983 -0.170489  -1.290364  -0.555154  -1.726133
513.0    1982 -0.227087  -0.920912  -0.600944  -2.180406
         1983 -3.662077 -16.374143 -10.755647 -34.634746
         1984 -0.686901  -6.027187  -1.934656  -6.332470
         1985  0.012776   0.120064   0.034938   0.110346
         1986  0.952121   8.897889   2.603646   8.073400
         1987  0.009403   0.087385   0.018431   0.074445
         1988 -0.596890  -5.516223  -1.765432  -5.311243
522.0    1980  0.298115   2.520544   0.887636   2.935602
         1981 -0.326324  -2.726662  -0.827140  -3.184516
...                 ...        ...        ...        ...
251235.0 1996  0.201721   2.056820   0.841265   2.303458
251388.0 1996  0.131144   1.304479   0.454535   1.326359
251433.0 1996  0.388793   3.636715   1.272274   4.229858
251478.0 1996 -0.460027  -5.374170  -2.091199  -6.142140
251676.0 1996  0.465239   4.255370   1.848341   5.264221
251712.0 1996 -0.143944  -1.713865  -0.506715  -1.908740
251748.0 1996  0.275914   2.498509   0.818525   2.647590
251829.0 1996 -1.418100 -15.611911  -5.100769 -16.445088
251838.0 1996 -0.511832  -4.119688  -1.567150  -5.599638
251847.0 1996  0.386301   2.323064   1.563449   4.261332
251874.0 1996  0.314886   2.931498   1.217191   3.436838
251883.0 1996 -0.918508  -7.707749  -2.908314  -9.769079
252081.0 1996  0.306938   2.093737   0.985321   3.347665
252207.0 1996  0.723769   6.714413   2.625686   8.301549
252315.0 1996 -1.034178  -9.090208  -3.380410 -11.755754
252324.0 1996  0.115376   0.851969   0.280359   1.203892
252513.0 1996  0.011276   0.123010   0.048692   0.140085
252666.0 1996 -1.085923  -8.673587  -2.689082 -11.472089
252882.0 1996 -0.173236  -2.154397  -0.730221  -2.232264
252918.0 1996 -0.183295  -2.214994  -0.778061  -2.313496
252963.0 1996 -2.082498 -17.361685  -7.001328 -22.864411
252972.0 1996  0.552257   6.886562   2.551292   7.600674
252981.0 1996 -0.282024  -3.408805  -1.027661  -3.137338
253215.0 1996  0.900966   8.413177   3.051814   9.553386
253224.0 1996 -0.367214  -2.783258  -1.193307  -3.814377
253269.0 1996  0.326241   4.295925   1.627307   4.311133
253341.0 1996 -1.272577 -16.336169  -4.816574 -17.414754
253431.0 1996  0.067214   0.635141   0.206752   0.698685
253593.0 1996  0.391553   4.585837   1.139065   4.408302
253629.0 1996 -1.733715 -22.236268 -11.119730 -26.091035

[18044 rows x 4 columns]

In [46]:
class ACF(GMM):
    
    def __init__(self, dta, stage1_resid, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        y,x = dta
        super(ACF, self).__init__(y,x,stage1_resid, x.shape[1] +2,*args, **kwds)
        self.endog = y
        self.exog = x
        self.instr = stage1_resids
        
    def momcond(self, params):
        d = self.endog, self.exog
        return np.array( stage2obj(params, self.instr , d) )
        
        
d = ldata['routput'],ldata[['rcapstock','totlab']]
acfmodel = ACF(d, stage1_resids)
acfresult = acfmodel.fit(np.array([1.3]+2*[.1]+[.5]))
print acfresult.summary()

         Current function value: 0.000411
         Iterations: 33
         Function evaluations: 208
         Gradient evaluations: 190
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 72
         Function evaluations: 82
         Gradient evaluations: 82
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1
                                 ACF Results                                  
Dep. Variable:                routput   Hansen J:                    5.656e-11
Model:                            ACF   Prob (Hansen J):                   nan
Method:                           GMM                                         
Date:                Wed, 15 May 2019 